In [48]:
import numpy as np
import pandas as pd

In [49]:
df_clv=pd.read_csv('df_clv.csv')
df_clv.head()

,age,gender,security_no,joining_date,joined_through_referral,referral_id,last_visit_time,days_since_last_login,avg_session_duration,avg_transaction_value,...,feedback_No reason specified,feedback_Poor Customer Service,feedback_Poor Product Quality,feedback_Poor Website,feedback_Products always in Stock,feedback_Quality Customer Care,feedback_Reasonable Price,feedback_Too many ads,feedback_User Friendly Website,Churn_probability
0,0.148148,0,XW0DQ7H,17-08-2017,0,xxxxxxxx,16:08:02,0.991220,0.514859,0.526717,...,0,0,0,0,1,0,0,0,0,0.003610
1,0.407407,0,5K0N3X1,28-08-2017,0,CID21329,12:38:13,0.990244,0.515803,0.121456,...,0,0,0,0,0,1,0,0,0,0.003547
2,0.629630,0,1F2TCL3,11-11-2016,1,CID12313,22:53:21,0.988293,0.550486,0.204074,...,0,0,0,1,0,0,0,0,0,0.964750
3,0.500000,1,VJGJ33N,29-10-2016,1,CID3793,15:57:50,0.985366,0.473971,0.246577,...,0,0,0,1,0,0,0,0,0,0.970951
4,0.388889,0,SVZXCWB,12-09-2017,0,xxxxxxxx,15:46:44,0.994146,0.483866,0.238950,...,0,0,0,1,0,0,0,0,0,0.959872


In [50]:
df_clv.dtypes

age                                               float64
gender                                              int64
security_no                                        object
joining_date                                       object
joined_through_referral                             int64
referral_id                                        object
last_visit_time                                    object
days_since_last_login                             float64
avg_session_duration                              float64
avg_transaction_value                             float64
avg_frequency_login_days                          float64
points_in_wallet                                  float64
used_special_discount                               int64
offer_application_preference                        int64
past_complaint                                      int64
churn                                               int64
logins_90d                                        float64
active_days_90

In [51]:
df_clv['joining_date']=pd.to_datetime(df_clv['joining_date'])
df_clv['last_visit_time']=pd.to_datetime(df_clv['last_visit_time'])

/var/folders/nc/hvlqvddd2xj11jl0l69nx3580000gn/T/ipykernel_42070/3519070926.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clv['joining_date']=pd.to_datetime(df_clv['joining_date'])
/var/folders/nc/hvlqvddd2xj11jl0l69nx3580000gn/T/ipykernel_42070/3519070926.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clv['last_visit_time']=pd.to_datetime(df_clv['last_visit_time'])


In [52]:
df_clv.dtypes

age                                                      float64
gender                                                     int64
security_no                                               object
joining_date                                      datetime64[ns]
joined_through_referral                                    int64
referral_id                                               object
last_visit_time                                   datetime64[ns]
days_since_last_login                                    float64
avg_session_duration                                     float64
avg_transaction_value                                    float64
avg_frequency_login_days                                 float64
points_in_wallet                                         float64
used_special_discount                                      int64
offer_application_preference                               int64
past_complaint                                             int64
churn                    

In [569]:
df_clv['transaction/day']=df_clv['avg_transaction_value']/(df_clv['avg_frequency_login_days']+0.1)
df_clv['transaction/day'].describe()

count    36992.000000
mean         0.483551
std          0.355919
min          0.000000
25%          0.218541
50%          0.437331
75%          0.657905
max          4.222199
Name: transaction/day, dtype: float64

In [570]:
abs(df_clv.drop(columns=['referral_id','security_no','CLV2','CLV']).corr()['churn']).sort_values(ascending=False)

churn                                             1.000000
Churn_probability                                 0.918877
plan_tier_Basic                                   0.726132
api_calls_90d                                     0.659219
active_days_90d                                   0.647458
logins_90d                                        0.643144
plan_tier_Enterprise                              0.606217
session_minutes_90d                               0.524295
points_in_wallet                                  0.292793
recency                                           0.285336
days_since_active                                 0.273094
transaction/day                                   0.234234
avg_transaction_value                             0.218012
feedback_Reasonable Price                         0.216665
feedback_User Friendly Website                    0.214590
feedback_Products always in Stock                 0.213867
feedback_Quality Customer Care                    0.2120

In [575]:
df_clv['CLV']=(df_clv['avg_frequency_login_days']*df_clv['points_in_wallet']*df_clv['avg_session_duration']*(1-df_clv['Churn_probability']))/((df_clv['days_since_active']+df_clv['days_since_last_login'])*df_clv['Churn_probability']+0.1)

In [532]:
df_clv['recency']=1/(1+df_clv['days_since_active'])

In [533]:
df_clv[['CLV','CLV2']]

,CLV,CLV2
0,4.758906,0.271979
1,0.234966,0.055474
2,0.000263,2.536862
3,0.000262,2.686343
4,0.000472,2.436291
...,...,...
36987,0.000307,2.901310
36988,0.000035,1.627394
36989,0.000742,4.658987
36990,0.002916,0.008629


In [576]:
df_clv[['CLV','CLV2']].describe()

,CLV,CLV2
count,36992.000000,36992.000000
mean,0.507146,1.536289
std,0.627305,1.859339
min,0.000000,0.000000
25%,0.003961,0.157638
50%,0.035487,0.416107
75%,1.158223,2.748209
max,3.219696,10.587178


In [577]:
abs(df_clv.drop(columns=['referral_id','security_no','CLV2']).corr()['CLV']).sort_values(ascending=False)

CLV                                               1.000000
Churn_probability                                 0.926775
churn                                             0.845975
api_calls_90d                                     0.728932
plan_tier_Enterprise                              0.701673
plan_tier_Basic                                   0.676113
logins_90d                                        0.659771
active_days_90d                                   0.659369
session_minutes_90d                               0.537684
points_in_wallet                                  0.354867
recency                                           0.283448
days_since_active                                 0.269852
avg_transaction_value                             0.230087
feedback_Quality Customer Care                    0.223979
feedback_Reasonable Price                         0.223856
feedback_Products always in Stock                 0.223366
transaction/day                                   0.2224

In [597]:
x_clv=df_clv.drop(columns=['referral_id','security_no','joining_date','CLV','CLV2','last_visit_time','churn','transaction/day','avg_frequency_login_days'])#,'avg_session_duration'])
y_clv=df_clv['CLV']

In [598]:
from sklearn.model_selection import train_test_split
x_train_clv,x_test_clv,y_train_clv,y_test_clv=train_test_split(x_clv,y_clv,test_size=0.2,random_state=42)

In [389]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


models={
    "LinearRegression":LinearRegression(),
    "RandomForest":RandomForestRegressor(n_estimators=200,random_state=42),
    "XGB":XGBRegressor(n_estimators=300,learning_rate=0.1,random_state=42),
    "LGBM":LGBMRegressor(n_estimators=300,learning_rate=0.1,random_state=42,verbose=-1),
    "CatBoost":CatBoostRegressor(iterations=300,learning_rate=0.1,verbose=False,random_state=42)
}


In [390]:
from sklearn.metrics import root_mean_squared_error,mean_absolute_error,r2_score

In [599]:
results={}

for name,model in models.items():
    model.fit(x_train_clv,y_train_clv)
    y_pred_clv=model.predict(x_test_clv)

    rmse=root_mean_squared_error(y_test_clv,y_pred_clv)
    mae=mean_absolute_error(y_test_clv,y_pred_clv)
    r2=r2_score(y_test_clv,y_pred_clv)

    results[name]={"RMSE":rmse,"MAE":mae,"R2_Score":r2}

In [600]:
results_df=pd.DataFrame(results)
results_df

,LinearRegression,RandomForest,XGB,LGBM,CatBoost
RMSE,0.198136,0.115210,0.113366,0.113384,0.112452
MAE,0.140630,0.056131,0.056067,0.056052,0.056852
R2_Score,0.898669,0.965740,0.966827,0.966817,0.967360


In [582]:
from sklearn.model_selection import cross_val_score
cross_val={}
for name,model in models.items():
    cross_validation=cross_val_score(model,x_train_clv,y_train_clv,cv=5,scoring='r2')
    cross_val[name]={"Cross_val_score":cross_validation.mean()}

In [583]:
scores_df=pd.DataFrame(cross_val)
scores_df

,LinearRegression,RandomForest,XGB,LGBM,CatBoost
Cross_val_score,0.88498,0.939204,0.937579,0.939126,0.940547


### So, CatBoost comes out to be best

In [584]:
clv_model=CatBoostRegressor(iterations=300,learning_rate=0.1,verbose=False,random_state=42)
clv_model.fit(x_train_clv,y_train_clv)
y_pred_clv_final=clv_model.predict(x_test_clv)
r2_score(y_test_clv,y_pred_clv_final)

0.9462739445689248